In [1]:
import pylab
from matplotlib import pyplot
import matplotlib as mpl
from pylab import *
from numpy import *
import pandas as pd
import sklearn
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC, SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from collections import OrderedDict
from operator import itemgetter
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from textblob.classifiers import NaiveBayesClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


ImportError: No module named 'pandas'

# Reading Dataset
## 1- The First Dataset
### The dataset is a folder contains two subfolders, which are positive and negative tweets.Each subfolder contains 1000 tweets, which are numbered in ascending order. "reading_datast" starts with initializing two empty arrays, Tarining_dataset and Training_labels.After that, there is a for loop for each subfolder which loops over the files.Reading is done using two built-in functions, The First one is openfile(filename, type) which save the file into a variable called "file". The second one is readlines(), which reads all the lines in the variable file and save it into a variable called "lines" which converted to a string and saved in a new variable "tweet".Another built-in function is used to remove redundant expressions.The last operation in this function is to append the actual tweet to "training_dataset" array, which is a substring from char 2 till the end except for the last 4 characters.The last 4 caharacters contains the lable of the tweet and was appended to "training_labels" array.


## reading_datast(void)--> returns tweets(array),labels(array). 

In [6]:
def reading_datast():
    training_labels=[]
    training_dataset=[]

    for x in range(1,1001):
        filename="twitter/positive/positive"+str(x)+".txt"
        file = open(filename, 'r')
        lines = file.readlines()
        tweet=str(lines)
        new_tweet=tweet.replace("\\xa0","")
        training_dataset.append(new_tweet[2:len(new_tweet)-4])
        training_labels.append("POS")
    for j in range(1,1001):
        if(j not in [103,116,176,178,180,184,186,189,191]):
            filename="twitter/negative/negative"+str(j)+".txt"
            file = open(filename, 'r')
            lines = file.readlines()
            tweet=str(lines)
            new_tweet=tweet.replace("\\xa0","")
            if(j in range(800)):
                training_dataset.append(new_tweet[2:len(new_tweet)-4])
                training_labels.append("NEG")
    return training_dataset,training_labels



# 2- ASTD:
### Reading_new_dataset function is for reading ASTD, which starts with initializing labels and dataset arrays.It also uses the built-in function openfile(filename,type) and saves the file in the "file" variable. After that, it loops over the lines and substring the line and appends the last 4 characters to the"labels" array. 
### There is a check to decide whether the label is neutral or not. If the label is "RAL" which means neutral then we append the whole line except for the last 8 characters, because the label is written as neutral not RAL which removes the "NEU" chars.If it is positive or negative or objective the whole line is added except for the last 5 chars, which contains the label itself and redundant expressions.There is another check if the label is equal to "OBJ", it will be changed to "RAL". 

## reading_new_dataset(filename(string))--> returns tweets(array),labels(array)

In [7]:
def reading_new_dataset(filename):
    labels = []
    dataset=[]
    file = open(filename, 'r')
    i=0
    for line in file :
        labels.append(line[-4:])
        labels[i]=labels[i][:3]
        if (labels[i]=="RAL"):
            dataset.append(line[:len(line)-8])
        else:
            dataset.append(line[:len(line)-5])
            if (labels[i]=="OBJ"):
                labels[i]="RAL"
        i=i+1

    return dataset,labels

## 3- Stop Words.
### An empty array "dataset" is initialized at the start of reading_stopwords. After that, the built-in function ("open(filename,type,encoding)") with the filename that was sent when reading_stopwors was called and encoding UTF-8.
### Then, there is a loop over the lines that appends each line of the stop words to the main array "dataset".

## reading_stopwords(filename(String))--> returns dataset(array)

In [8]:
def reading_stopwords(filename):
    dataset=[]
    file = open(filename, 'r',encoding='utf-8')
    i=0
    for line in file :
        dataset.append(line[:-1])
    return dataset

# Datset Manipulation
### Reading_new_dataset function is for reading ASTD, which starts with initializing labels and dataset arrays.It also uses the built-in function openfile(filename,type) and saves the file in the "file" variable. After that, it loops over the lines and substring the line and appends the last 4 characters to the"labels" array. 
### There is a check to detrmine  whether the label is objective is will be appended to labels array as it, anything else will be appended as "subjective".

## reading_and_manipulation(filename(string))--> returns tweets(array),labels(array)

In [9]:
def reading_and_manipulation(filename):
    labels = []
    dataset=[]
    file = open(filename, 'r')
    i=0
    for line in file :
        line=file.readline()
        labels.append(line[-4:])
        labels[i]=labels[i][0:3]
        if (labels[i]=="OBJ"):
            labels[i]="OBJ"
        else:
            labels[i]="SUB"
            dataset.append(line[0:len(line)-5])
        i=i+1
    return dataset,labels

##  Remove Hashtags
### Remove_hashtags starts with initializing an array "words_without_hashtags", then it loops over an array of words which has been called with. There are multiple checks here, the first one checks whether the word contains a hashtag or not.If there is a hashtag, the built- in function split(argument) will be used, which returns a list of strings without the argument and saved in an array "hashtag_word".
### The second check will be on "hashtag_word" , if the second element contains "_" or not. If it contains "_" another split will be done and saved in "two_words" array to loop over them and each word will be added to the main array "words_without_hashtags". Else, if it does not contain the "_" then the second item will be just one word and will be added to automatically  to the main array "words_without_hashtags".

## remove_hashtag(words(array))--> returns words_woithout_hashtag(array).

In [10]:
def remove_hashtag (words):
    words_without_hashtags=[]
    for word in words:
        if "#" in word:
            hashtag_word=word.split("#")
            if "_" not in hashtag_word[1]:
                words_without_hashtags.append(hashtag_word[1])
            else:
                two_words=hashtag_word[1].split("_")
                for new_word in two_words:
                    words_without_hashtags.append(new_word)
        else:
            words_without_hashtags.append(word)
    return words_without_hashtags

## Remove Punctuation
### Remove_punctutaion starts with initializing an array "new_words_without_punctuation" .Then, it loops over an array of words that it called with to determine if a word contains a "," or not. If a word contains it, it calls the built-in function "split(",")" and saved the returned list of a word without punctuation in an array "new_words". 
### Another check aims to check if "," comes before the word or after it by check the length of the first element of new_word. If the length is greater than 1 , then the second element will be added to the main array "new_words_without_punctuation". Otherwise, it means that it comes after the word itself, then the first element will be chosen to be added to the main array.If the word does not contain "," at all, it will be added directly to the main array. 

## remove_punctuation(words(array))--> returns new_words_without_punctuation(array)

In [11]:
def remove_punctuation(words):
    new_words_without_punctuation=[]
    for word in words:
        if "،" in word:
            new_word=word.split("،")
            if(len(new_word[0]) > 1):
                new_words_without_punctuation.append(new_word[0])
            else:
                new_words_without_punctuation.append(new_word[1])
        else:
            new_words_without_punctuation.append(word)
    
    return new_words_without_punctuation

## Clean Tweet
### Clean_tweet takes as an input the whole tweet and then returned the words of the tweets without hashtags or punctuation.First of all, it split the whole tweet by calling the built-in function "split("spaces")", which splits the tweet by spaces and save the returned list in an array "remove_spaces" . 
### After that, remove_hashtags function is called with "remove_spaces" and the returned output saved in an array "new_wrods_without_hashtags".The final step is sending the "new_words_without_hashtags" to remove_punctuation function and save the returned output in an array "cleaned", which is the main array.

## Clean_tweet(tweet(string))-->returns(cleaned_tweet(array))

In [12]:
def clean_tweet(tweet):
    remove_spaces=tweet.split(" ")
    new_words_without_hashtags=remove_hashtag(remove_spaces)
    cleaned=remove_punctuation(new_words_without_hashtags)
    return cleaned

## Frequency Calculation

frequency_calc a function that takes tweets as an input and starts with initializing an empty dictionary, which will contain the frequency of each word.Then, it continues with looping over the tweets and calls clean_tweet for each tweet and save the returned array in array "cleaned".After that, it loops over the "cleaned" array to determine whether each the "key" dictionary contains this word or not. If it contains the word, it will increment the number of occurrences of this word and breaks the loop over the divisionary.
Otherwise, it will add it to the dictionary with value equals to 1.

## frequency_calc(tweets(array))--> returns(words(dictionary)).

In [13]:
def frequency_calc (tweets):
    words={}
    for tweet in tweets:
        cleaned_tweet=clean_tweet(tweet)
        for word in cleaned_tweet:
            flag=True
            for key in words:
                if word==key:
                    words[key]+=1
                    flag=False
                    break
            if flag:
                words[word]=1
    return words

In [14]:
def frequently_used (words):
    sorted_words = [(k, words[k]) for k in sorted(words, key=words.get, reverse=True)]
    for k, v in sorted_words:
        k, v
    return sorted_words

## With_stopwords
with_stopwords it takes tweets as an input, then it starts with calling frequency_calc and saves the returned output in a dictionary "words".
After that, it calls "frequently_used" with "words" and saves the returned output in an array "sorted_words", which will be returned.

## with_stopwords(tweets(array))--> returns(sorted_bigram_words(array))

In [15]:
def with_stopwords(tweets):
    words=frequency_calc(tweets)
    sorted_words=frequently_used(words)
    return sorted_words

## Without_Stopwords

wihout_stopwords it also takes tweets as an input, then it starts with calling frequency_calc and saves the returned output in a dictionary "words", Yet, there is a difference here. It calls reading_stopwords and saves the returned output in an array "stopwords". Then, it loops over the dictionary"words" and checks whether it contains a stop word or not. If it contains a stop word, it changes its value to -1. Then it calls frequently_used with the modified array. and save the returned output in sorted_words, which will be returned.

## without_stopwords(tweets(array))--> returns(sorted_words_without_stop_words(array))

In [16]:
def without_stopwords(tweets):
    words=frequency_calc(tweets)
    stopwords=reading_stopwords("/Users/ahmed/Desktop/Bachelor/stopwords.txt")
    for word in words:
        if word in stopwords:
            words[word]=-1
    sorted_words=frequently_used(words)
    return sorted_words	

## Bigram Words

The two_gram function starts with initializing an empty dictionary "words" which will be used to store the bigram words with its frequencies.It takes as an input the whole tweets and starts looping over them and calls clean_tweet function for each tweet of them and store the returned cleaned tweet in an array "cleaned_tweet".
It continues to loop over the "cleaned_tweet" array.Then, it takes each word with its neighbor and saves the bigram words in an array "two_gram_tweet".After that, it loops over the dictionary to count occurrences of each bigram word.
## two_gram(tweets(array))--> returns(sorted_bigram_words(array))

In [17]:
def two_gram(tweets):
    words={}
    for tweet in tweets:
        cleaned_tweet=clean_tweet(tweet)
        two_gram_tweet=[]
        for x in range(len(cleaned_tweet)-1):
            two_gram_tweet.append(cleaned_tweet[x]+" "+cleaned_tweet[x+1])
        for word in two_gram_tweet:
            flag=True
            for key in words:
                if word==key:
                    words[key]+=1
                    flag=False
                    break
            if flag:
                words[word]=1
                sorted_words=frequently_used(words)
    return sorted_words

## Trigram Words

The three_gram function starts with initializing an empty dictionary "words" which will be used to store the trigram words with its frequencies.It takes as an input the whole tweets and starts looping over them and calls clean_tweet function for each tweet of them and store the returned cleaned tweet in an array "cleaned_tweet".
It continues to loop over the "cleaned_tweet" array.Then, it takes each word with its neighbors and saves the trigram words in an array "three_gram_tweet".After that, it loops over the dictionary to count occurrences of each trigram word.

## Three_gram(tweets(array))--> returns(sorted_trigram_words(array))

In [18]:
def three_gram(tweets):
    words={}
    for tweet in tweets:
        cleaned_tweet=clean_tweet(tweet)
        three_gram_tweet=[]
        for x in range(len(cleaned_tweet)-1):
            if((x+2) in range (len(cleaned_tweet))):
                three_gram_tweet.append(cleaned_tweet[x]+" "+cleaned_tweet[x+1]+" "+cleaned_tweet[x+2])

        for word in three_gram_tweet:
            flag=True
            for key in words:
                if word==key:
                    words[key]+=1
                    flag=False
                    break
            if flag:
                words[word]=1
    sorted_words=frequently_used(words)
    return sorted_words

## Feature extraction
The feature_extraction function takes tweets, words and a limit to the words as inputs.It starts to take the limited words and save it in "most_frequently_used" array. Then, an empty array is initialized "features" which is an array of arrays of booleans. Then, it continues with looping over the tweets and determine if a word in the tweet or not and if its length is bigger than 1 or not to reduce the number of redundant expressions. If the word in the tweet a true boolean flag is added to the flags array. After finishing the tweet's loop, its boolean flags is added to features.

### feature_extraction(tweets(array),words(dictionary),limit(int))-->returns features((an array of arrays))

In [19]:
def feature_extraction(tweets,words,limit):
    most_frequently_used=words[:limit]
    features=[]
    for tweet in tweets:
        flags=[]
        for word in most_frequently_used:
            if len(word[0]) >1 and word[1] != -1 : 
                if word[0] in tweet :
                    flags.append(1)
                else:
                    flags.append(0)
        features.append(flags)
    
    return features

In [20]:
def setup():
    new_dataset=reading_new_dataset("/Users/ahmed/Desktop/Bachelor/ASTD-master/data/tweets.txt")
    new_tweets=new_dataset[0]
    new_lables=new_dataset[1]
    words_without=without_stopwords(new_tweets)
    print("words_without")
    words=with_stopwords(new_tweets)
    print("words")
    two_gram_words=two_gram(new_tweets)
    print("two_gram")
    three_gram_words=three_gram(new_tweets)
    print("three_gram")
    words_two_gram=words+two_gram_words
    words_three_gram=words+three_gram_words
    words_all=words+two_gram_words+three_gram_words
    
    words_without_two_gram=words_without+two_gram_words
    words_without_three_gram=words_without+three_gram_words
    words_without_all=words_without+two_gram_words+three_gram_words
    
    
    features_words=feature_extraction(new_tweets,words,len(words))
    print("features_words")
    
    features_words_without=feature_extraction(new_tweets,words_without,len(words_without))
    print("features_words_without")
    
    features_two_gram_words=feature_extraction(new_tweets,two_gram_words,len(two_gram_words))
    print("features_two_gram_words")
    
    features_three_gram_words=feature_extraction(new_tweets,three_gram_words,len(three_gram_words))
    print("features_three_gram_words")
    
    features_words_two_gram=feature_extraction(new_tweets,words_two_gram,len(words_two_gram))
    print("features_words_two_gram")
    
    features_words_three_gram=feature_extraction(new_tweets,words_three_gram,len(words_three_gram))
    print("features_words_three_gram")
    
    features_words_all=feature_extraction(new_tweets,words_all,len(words_all))
    print("features_words_all")

    features_words_without_two_gram=feature_extraction(new_tweets,words_without_two_gram,len(words_without_two_gram))
    print("features_words_without_two_gram")
    
    features_words_without_three_gram=feature_extraction(new_tweets,words_without_three_gram,len(words_without_three_gram))
    print("features_words_without_three_gram")
    
    features_words_all=feature_extraction(new_tweets,words_without_all,len(words_without_all))
    print("features_words_all")

    return new_tweets,new_lables,words_without,words,two_gram_words,three_gram_words,features_words,features_words_without,features_two_gram_words,features_three_gram_words,features_words_two_gram,features_words_three_gram,features_words_all,features_words_without_two_gram,features_words_without_three_gram,features_words_all


In [ ]:
sets=setup()

words_without
words


In [3]:
def main():
    new_tweets=sets[0]
    new_labels=sets[1]
    words_without=sets[2]
    words=sets[3]
    two_gram_words=sets[4]
    three_gram_words=sets[5]
    features_words=sets[6]
    features_words_without=sets[7]
    features_two_gram_words=sets[8]
    features_three_gram_words=sets[9]
    features_words_two_gram=sets[10]
    features_words_three_gram=sets[11]
    features_words_all=sets[12]
    features_words_without_two_gram=sets[13]
    features_words_without_three_gram=sets[14]
    features_words_all=sets[15]
    file=open("gaussian.txt","w")
    
    
    words_gaussian_score=gaussian_class(features_words,new_labels)
    file.write(words_gaussian_score+"words_gaussian_score\n")
    print("1")
    
    words_without_gaussian_score=gaussian_class(features_words_without,new_labels)
    file.write(words_without_gaussian_score+"words_without_gaussian_score\n")
    print("2")
    
    
    two_gram_words_gaussian_score=gaussian_class(features_two_gram_words,new_labels)
    file.write(two_gram_words_gaussian_score+"two_gram_words_gaussian_score\n")
    print("3")
    
    three_gram_words_gaussian_score=gaussian_class(features_two_gram_words,new_labels)
    file.write(three_gram_words_gaussian_score+"three_gram_words_gaussian_score\n")
    print("4")
    
    words_two_gram_gaussian_score=gaussian_class(features_words_two_gram,new_labels)
    file.write(words_two_gram_gaussian_score+"words_two_gram_gaussian_score\n")
    print("5")
    
    words_three_gram_words_gaussian_score=gaussian_class(features_words_three_gram,new_labels)
    file.write(words_three_gram_words_gaussian_score+"words_three_gram_words_gaussian_score\n")
    print("6")
    
    words_all_gaussian_score=gaussian_class(features_words_all,new_labels)
    file.write(words_all_gaussian_score+"words_all_gaussian_score\n")
    print("7")
    
    words_without_two_gram_gaussian_score=gaussian_class(features_words_without_two_gram,new_labels)
    file.write(words_without_two_gram_gaussian_score+"words_without_two_gram_gaussian_score\n")
    print("8")
    
    words_without_three_gram_gaussian_score=gaussian_class(features_words_without_three_gram,new_labels)
    file.write(words_without_three_gram_gaussian_score+"words_without_three_gram_gaussian_score\n")
    print("9")
    
    words_without_all_gaussian_score=gaussian_class(features_words_all,new_labels)
    file.write(words_without_all_gaussian_score+"words_without_all_gaussian_score\n")
    print("10")
    
    
    file.close()

In [4]:
main()

NameError: name 'sets' is not defined

## Gaussian

In [38]:
def gaussian_class(features,labels):
    
    training_labe=labels[:8000]
    testing_labe=labels[-2006:]
    training_datas=features[:8000]
    testing_datas=features[-2006:]
    
    gclassfier= GaussianNB()
    gclassfier.fit(training_datas,training_labe)
    gscore = gclassfier.score(testing_datas, testing_labe)
    
    return (gscore)

In [39]:
def multinomial_class(features,labels):
    
    training_labe=labels[:8000]
    testing_labe=labels[-2006:]
    training_datas=features[:8000]
    testing_datas=features[-2006:]
    
    classfier= MultinomialNB()
    classfier.fit(training_datas,training_labe)
    score = classfier.score(testing_datas, testing_labe)
    
    return(score)

In [40]:
def bernoulli_class(features,labels):
    
    training_labe=labels[:8000]
    testing_labe=labels[-2006:]
    training_datas=features[:8000]
    testing_datas=features[-2006:]
    
    classfier= BernoulliNB()
    classfier.fit(training_datas,training_labe)
    score = classfier.score(testing_datas, testing_labe)
    
    return(score)

In [41]:
def svm_class(features,labels):
    
    training_labe=labels[:8000]
    testing_labe=labels[-2006:]
    training_datas=features[:8000]
    testing_datas=features[-2006:]
    
    classfier= svm=sklearn.svm.SVC()
    classfier.fit(training_datas,training_labe)
    score = classfier.score(testing_datas, testing_labe)
    
    return(score)

In [42]:
def randomforest_class(features,labels):
    
    training_labe=labels[:8000]
    testing_labe=labels[-2006:]
    training_datas=features[:8000]
    testing_datas=features[-2006:]
    
    classfier= RandomForestClassifier()
    classfier.fit(training_datas,training_labe)
    score = classfier.score(testing_datas, testing_labe)
    
    return(score)

In [43]:
def class_NB_univariant(features,labels,number):

    new_features = SelectKBest(chi2, k=number).fit_transform(features,labels)
    
    training_labels=labels[:1598]
    testing_labels=labels[-402:]
    training_dataset=new_features[:1598]
    testing_dataset=new_features[-402:]
    #print(testing_labels)
    classfier= OneVsRestClassifier(MultinomialNB())
    classfier.fit(training_dataset,training_labels)
    score = classfier.score(testing_dataset, testing_labels)
    return score
